# Developing an ML Model End-to-End: Exercise Solutions

Now that you've had some exposure to using Python for training, tuning, evaluating, and selecting an ML model, it's time to try some exercises on your own. Below, you'll find four exercises that utilize a new training data set, the Diagnostic Breast Cancer Wisconsin data set. You can read more about this data set [here](https://archive.ics.uci.edu/ml/datasets/breast+cancer+wisconsin+(diagnostic)). When applicable, an exercise will begin with some starter code to help guide your development towards a solution to the problem. Remember, there isn't just one solution to the problem and your code may produce the similar results even if it's written differently from what is supplied in the solution notebook.

These exercises are intended to be performed on Anaconda Notebooks to avoid the need to download any additional Python libraries besides `ydata_profiler`. If you decide to do these exercises on your own machine, you'll need to download the data set locally as well as any of the Python libraries used. The first code block is provided for you to download the data set.

In [ ]:
# Import Python libraries needed
import pandas as pd
import os
import urllib
import tarfile

# !pip install ydata-profiling
 
# Create a function for pulling U.S. Census data for California housing
DOWNLOAD_ROOT = 'https://raw.githubusercontent.com/jsukup/Developing-an-ML-Model-End-to-End/master'
DATA_PATH = os.path.join('datasets', 'cancer')
DATA_URL = DOWNLOAD_ROOT + 'datasets/cancer/cancer.tgz'

def fetch_cancer_data(data_url=DATA_URL, data_path=DATA_PATH):
    if not os.path.isdir(data_path):
        os.makedirs(data_path)
    tgz_path = os.path.join(data_path, 'cancer.tgz')
    urllib.request.urlretrieve(data_url, tgz_path)
    data_tgz = tarfile.open(tgz_path)
    data_tgz.extractall(path=data_path)
    data_tgz.close()

fetch_cancer_data() # Pull data set from GitHub

# Create a function for loading data into a Python object
def load_cancer_data(data_path=DATA_PATH):
    csv_path = os.path.join(data_path, 'cancer.csv')
    return pd.read_csv(csv_path)

data = load_cancer_data()
data.head() # Inspect first five rows of data set

## Exercise 1: Describe and Verify Data Quality

To begin, have a look at the data set in its raw form before performing any transformations. The easiest approach is the use the `ydata_profiling` library.  Answer the following questions:
1. Describe the data set. How many variables/features does it include? What do you believe is the target we would like to predict?
2. Are there any missing values?
3. Are there any highly correlated variables/features with a correlation coefficient >.7? If so, what are they? Are there any variables/features highly correlated with the target variable that might be good predictors?
4. Are there any variables/features we can remove?
5. What is the proportion of each label for the target variable/feature?

In [ ]:
# Run the data profiler and save the output to an HTML file
from ydata_profiling import ProfileReport 

profile = ProfileReport(data)
profile.to_file('profile.html')

## Exercise 2: Data Preparation

1. Write a function to delete any variables/features that are not useful to training the model (e.g., empty variables/features)
2. Write a function to convert the categorical target variable/feature to a binary, numeric one.
3. Write a function to apply `StandardScaler` to the training variables/features (but *not* the target variable/feature)

In [ ]:
# Function to delete any unneeded variables/features
def delete_unnamed_32(data):
    if 'Unnamed: 32' in data.columns:
        return data.drop('Unnamed: 32', axis=1, inplace=True)
    else:
        return print("Variable 'Unnamed: 32' does not exist in the DataFrame.")

delete_unnamed_32(data)

# Function to convert the categorical target variable/feature
def convert_target_variable(data, target_column):
    data[target_column] = data[target_column].map({'B': 0, 'M': 1})
    return data

convert_target_variable(data, 'diagnosis')

# Function to apply `StandardScaler`
from sklearn.preprocessing import StandardScaler

def apply_standard_scaler(data):
    scaler = StandardScaler()
    data.iloc[:,2:] = scaler.fit_transform(data.iloc[:,2:])
    return data

apply_standard_scaler(data)

## Exercise 3: Model Development

1. Split the data set into training data and testing data using the `scikit-learn` function `train_test_split`. You should create 4 new Python objects: `x_train`,`x_test`,`y_train`,`y_test`. This way we'll hold the target variable/feature in it's own object. Use an 80/20 train/test split with the `random_state=734`. 

In [ ]:
# Create a train/test split
from sklearn.model_selection import train_test_split 

y = data['diagnosis']
x = data.drop(['id','diagnosis'], axis=1)
x_train, x_test, y_train, y_test = train_test_split(x,
                                                    y,
                                                    test_size=0.2,
                                                    random_state=734) 

2. Train up to 3 different models using `scikit-learn` to build a model that predicts the target variable/feature using the training variables/features. Since the target is a discrete value, this will be a classification model. More information on models and `scikit-learn` can be found [here](https://scikit-learn.org/stable/). After training, save each model's prediction on the testing data in a new Python object. You don't have to adjust the default hyperparameters.

In [ ]:
# Logistic Regression model
from sklearn.linear_model import LogisticRegression

lr_clf = LogisticRegression()
lr_clf.fit(x_train, y_train)

# Random Forest model
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier()
rf_clf.fit(x_train, y_train)

# K-Nearest Neighbors model
from sklearn.linear_model import SGDClassifier

sgd_clf = SGDClassifier()
sgd_clf.fit(x_train, y_train)

3. Evaluate each model using a simple measurement of Accuracy (use the `scikit-learn` method `.score()`) comparing the predicted values with the actual values in the `y_test` Python object created earlier. 

In [ ]:
# Model accuracy
lr_score = lr_clf.score(x_test, y_test)
rf_score = rf_clf.score(x_test, y_test)
sgd_score = sgd_clf.score(x_test, y_test)

print(f"Model accuracy:\n Logistic regression: {lr_score},\n Random Forest: {rf_score},\n SGD: {sgd_score}")

## Exercise 4: Hyperparameter Optimization and Model Selection

Now that we've trained a few candidate models, we can try and improve their performance using hyperparameter optimization.

1. Chose a single model or all the candidate models to perform hyperparameter optimization on. You may use the built-in tools in `scikit-learn` that we used in the course exercises (e.g., `GridSearchCV` and `RandomizedSearchCV`). Remember to choose a `scoring` parameter that's suitable for a classification model like `accuracy`! You should be able to reuse a lot of the same code from the course lab, too.

In [ ]:
# Train a Random Forest model using grid search with cross-validation
from sklearn.model_selection import GridSearchCV

# Create a list of hyperparameter values set as key:value pairs
param_grid = [
    # Try 12 (3×4) combinations of hyperparameters
    {'n_estimators': [3, 5, 10, 30], 'max_features': [2, 4, 6, 8, 10]},
    # Try 6 (2×3) combinations with bootstrap set as False
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
]

# Train Random Forest model
forest_clf = RandomForestClassifier(random_state=734)

# Five fold cross-validation: (12+6)*5=90 rounds of training 
grid_search = GridSearchCV(forest_clf, 
                           param_grid, 
                           cv=5,
                           scoring='accuracy',
                           return_train_score=True)

# Fit model to training data
grid_search.fit(x_train, y_train)

# Print best parameters and estimator
print(f"Optimal estimated hyperparameters: {grid_search.best_estimator_}")

In [ ]:
# Train a Random Forest model using grid search with cross-validation
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

# Create a range of hyperparameter values set as key:value pairs
param_distribs = {
        'n_estimators': randint(low=1, high=200),
        'max_features': randint(low=1, high=8),
}

# Train Random Forest model
forest_clf = RandomForestClassifier(random_state=734)

# Five fold cross-validation: (12+6)*5=90 rounds of training 
rand_search = RandomizedSearchCV(forest_clf, 
                                 param_distribs,
                                 cv=5,
                                 scoring='accuracy',
                                 return_train_score=True)

# Fit model to training data
rand_search.fit(x_train, y_train)

# Print best parameters and estimator
print(f"Optimal estimated hyperparameters: {rand_search.best_estimator_}")

2. Compare the results of the first model(s) to the second model(s) with optimal hyperparameters. You'll probably have different results depending on which technique was used. Which model performs the best?

In [ ]:
# Train a Random Forest model with GridSearch optimal hyperparameters
forest_gs = RandomForestClassifier(max_features=8, n_estimators=10, random_state=734)
forest_gs.fit(x_train, y_train)
forest_gs_score = forest_gs.score(x_test, y_test)

# Train a Random Forest model with the RandomSearch optimal hyperparameters
forest_rs = RandomForestClassifier(max_features=5, n_estimators=168, random_state=734)
forest_rs.fit(x_train, y_train)
forest_rs_score = forest_rs.score(x_test, y_test)

# Print the results
print(f"Model accuracy:\n Random Forest (GS): {forest_gs_score},\n Random Forest (RS): {forest_rs_score},\n Random Forest (default): {rf_score}")


# Congrats! That's the end of the exercises!